(f_rs_extraction)=


---------------
```{admonition} Learning Objectives
  - Subset bands by index or name
  - Extract raster data by row and column number
  - Extract data by bounding window
  - Extract raster data by coordinates
  - Extract raster data by geometry (point, polygon)
```
```{admonition} Review
* [Data Structures](c_features.md)
* [Raster Data ](c_rasters.md)
* [Reading and writing remotely sensed data ](f_rs_io.md)
```
--------------


# Raster Data Extraction
Raster data is often of little use unless we can extract and summarize the data. For instance, extracting raster to points by coordinates allows us to pass data to machine learning models for land cover classification or cloud masking. 

## Subsetting rasters
We can subset sections of the data array in a number of ways. In this case we will create a slice based on row and column location to subset LandSat data using a `rasterio.window.Window`.

Either a `rasterio.window.Window` object or tuple can be used with `geowombat.open`.

In [27]:
import rioxarray as rxr
import xarray as xr
import numpy as np
from rasterio.windows import Window
import rasterio
from rasterio.transform import from_bounds

# Create sample RGB data similar to rgbn
height, width = 500, 500
rgbn_data = np.random.randint(0, 255, (3, height, width), dtype=np.uint8)
transform = from_bounds(-120, 35, -119, 36, width, height)

# Save as temporary file
rgbn = '/tmp/rgbn.tif'
with rasterio.open(rgbn, 'w', driver='GTiff', height=height, width=width,
                   count=3, dtype=rgbn_data.dtype, crs='EPSG:4326',
                   transform=transform) as dst:
    dst.write(rgbn_data)

w = Window(row_off=0, col_off=0, height=100, width=100)
src = rxr.open_rasterio(rgbn, chunks=True)
src = src.sel(band=[1, 2, 3]).astype('float32')
src = src.assign_coords(band=['blue', 'green', 'red'])
src = src.isel(x=slice(w.col_off, w.col_off + w.width), 
               y=slice(w.row_off, w.row_off + w.height))
print(src)


<xarray.DataArray (band: 3, y: 100, x: 100)> Size: 120kB
dask.array<getitem, shape=(3, 100, 100), dtype=float32, chunksize=(1, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 800B -120.0 -120.0 -120.0 ... -119.8 -119.8 -119.8
  * y            (y) float64 800B 36.0 36.0 36.0 35.99 ... 35.81 35.8 35.8 35.8
    spatial_ref  int64 8B 0
  * band         (band) <U5 60B 'blue' 'green' 'red'
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0


We can also slice a subset of data using a tuple of bounded coordinates.

``` python
bounds = (793475.76, 2049033.03, 794222.03, 2049527.24)

with gw.open(rgbn,
                band_names=['green', 'red', 'nir'],
                num_workers=8,
                indexes=[2, 3, 4],
                bounds=bounds,
                out_dtype='float32') as src:
    print(src)
```

The configuration manager provides an alternative method to subset rasters. See `tutorial-config` for more details.

``` python
with gw.config.update(ref_bounds=bounds):

    with gw.open(rgbn) as src:
        print(src)
```

By default, the subset will be returned by the upper left coordinates of the bounds, potentially shifting cell alignment with the reference raster. To subset a raster and align it to the same grid, use the **ref_tar** keyword. This is equivalent to a "snap raster" in ArcGIS. 

``` python
with gw.config.update(ref_bounds=bounds, ref_tar=rgbn):

    with gw.open(rgbn) as src:
        print(src)
```

## Extracting data by coordinates
 
To extract values at a coordinate pair, translate the coordinates into array indices. For extraction by geometry, for instance with a shapefile, see [extract by point geometry](f_rs_extraction_point).

In [28]:
import rioxarray as rxr
import rasterio
import numpy as np
from rasterio.transform import from_bounds

# Create sample Landsat data
height, width = 1000, 1000
landsat_data = np.random.randint(0, 4096, (7, height, width), dtype=np.uint16)
transform = from_bounds(500000, 4000000, 530000, 4030000, width, height)  # UTM coordinates

l8_224078_20200518 = '/tmp/l8_224078_20200518.tif'
with rasterio.open(l8_224078_20200518, 'w', driver='GTiff', height=height, width=width,
                   count=7, dtype=landsat_data.dtype, crs='EPSG:32633',
                   transform=transform) as dst:
    dst.write(landsat_data)

# Coordinates in map projection units - use center of raster
with rasterio.open(l8_224078_20200518) as dataset:
    bounds = dataset.bounds
    y, x = (bounds.bottom + bounds.top) / 2, (bounds.left + bounds.right) / 2
    # Transform the map coordinates to data indices
    j, i = dataset.index(x, y)
    # Check if indices are within bounds
    if 0 <= j < dataset.height and 0 <= i < dataset.width:
        data = dataset.read()[:, j, i]
    else:
        print(f"Coordinates ({x}, {y}) are out of bounds")
        data = np.full(dataset.count, dataset.nodata or 0)
print(data.flatten())

[1769 1062 2295   53  921 2185 1867]


A latitude/longitude pair can be extracted after converting to the map projection.

In [29]:
import rioxarray as rxr
import rasterio
from pyproj import Transformer
import numpy as np
from rasterio.transform import from_bounds

# Use the same sample data
height, width = 1000, 1000
landsat_data = np.random.randint(0, 4096, (7, height, width), dtype=np.uint16)
transform = from_bounds(500000, 4000000, 530000, 4030000, width, height)

l8_224078_20200518 = '/tmp/l8_224078_20200518.tif'
with rasterio.open(l8_224078_20200518, 'w', driver='GTiff', height=height, width=width,
                   count=7, dtype=landsat_data.dtype, crs='EPSG:32633',
                   transform=transform) as dst:
    dst.write(landsat_data)

# Get actual center coordinates in lat/lon
with rasterio.open(l8_224078_20200518) as dataset:
    bounds = dataset.bounds
    center_x, center_y = (bounds.left + bounds.right) / 2, (bounds.bottom + bounds.top) / 2
    transformer = Transformer.from_crs(dataset.crs, "EPSG:4326", always_xy=True)
    lon, lat = transformer.transform(center_x, center_y)

with rasterio.open(l8_224078_20200518) as dataset:
    # Transform the coordinates to map units
    transformer = Transformer.from_crs("EPSG:4326", dataset.crs, always_xy=True)
    x, y = transformer.transform(lon, lat)
    # Transform the map coordinates to data indices
    j, i = dataset.index(x, y)
    # Check if indices are within bounds
    if 0 <= j < dataset.height and 0 <= i < dataset.width:
        data = dataset.read()[:, j, i]
    else:
        print(f"Coordinates ({x}, {y}) are out of bounds")
        data = np.full(dataset.count, dataset.nodata or 0)
print(data.flatten())

[2781 1094  746 4067 3429 2953 1786]


(f_rs_extraction_point)=
## Extracting data with point geometry 

In the example below, 'l8_224078_20200518_points' is a [GeoPackage](https://www.geopackage.org/) of point locations, and the output `df` is a [GeoPandas GeoDataFrame](https://geopandas.org/docs/reference/api/geopandas.GeoDataFrame.html?highlight=geodataframe#geopandas.GeoDataFrame). To extract the raster values at the point locations, use `geowombat.extract`.

In [30]:
import rioxarray as rxr
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

# Create sample points
points = [Point(510000, 4010000), Point(520000, 4020000)]
l8_224078_20200518_points = gpd.GeoDataFrame(
    {'id': [1, 2], 'name': ['point1', 'point2']}, 
    geometry=points, 
    crs='EPSG:32633'
)

src = rxr.open_rasterio(l8_224078_20200518, chunks=True)
df = src.rio.clip_box(*l8_224078_20200518_points.total_bounds, crs=l8_224078_20200518_points.crs)
print(df)

<xarray.DataArray (band: 7, y: 334, x: 334)> Size: 2MB
dask.array<getitem, shape=(7, 334, 334), dtype=uint16, chunksize=(1, 334, 334), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 56B 1 2 3 4 5 6 7
  * x            (x) float64 3kB 5.1e+05 5.1e+05 5.101e+05 ... 5.2e+05 5.2e+05
  * y            (y) float64 3kB 4.02e+06 4.02e+06 ... 4.01e+06 4.01e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0


```{note} 

The line `df = src.gw.extract(l8_224078_20200518_points)` could also have been written as `df = gw.extract(src, l8_224078_20200518_points)`.
```

In the previous example, the point vector had a CRS that matched the raster (i.e., EPSG=32621, or UTM zone 21N). If the CRS had not matched, the `geowombat.extract` function transforms the CRS on-the-fly.

In [31]:
import rioxarray as rxr
import geopandas as gpd
import rasterio
from shapely.geometry import Point
import pandas as pd

# Create sample points
points = [Point(510000, 4010000)]
l8_224078_20200518_points = gpd.GeoDataFrame(
    {'id': [1], 'name': ['point1']}, 
    geometry=points, 
    crs='EPSG:32633'
)

point_df = l8_224078_20200518_points.copy()
print(point_df.crs)
# Transform the CRS to WGS84 lat/lon
point_df = point_df.to_crs('epsg:4326')
print(point_df.crs)

# Extract values at point locations using rasterio sampling
results = []
with rasterio.open(l8_224078_20200518) as src:
    # Transform points back to raster CRS for sampling
    points_in_raster_crs = point_df.to_crs(src.crs)
    
    for idx, point in points_in_raster_crs.iterrows():
        geom = point.geometry
        if geom.geom_type == 'Point':
            row, col = src.index(geom.x, geom.y)
            if 0 <= row < src.height and 0 <= col < src.width:
                values = src.read()[:, row, col]
                result = {f'band_{i+1}': val for i, val in enumerate(values)}
                result.update({col: point[col] for col in point_df.columns if col != 'geometry'})
                results.append(result)

df = pd.DataFrame(results)
print(df)

EPSG:32633
epsg:4326
   band_1  band_2  band_3  band_4  band_5  band_6  band_7  id    name
0    3957    3323      88     997    3503    3724    3133   1  point1


Set the data band names using `sensor = 'bgr'`, which assigns the band names blue, green, red.

In [37]:
import geopandas as gpd
from shapely.geometry import Point
import rasterio

# Open raster
raster_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"
with rasterio.open(raster_path) as src:
    bounds = src.bounds
    crs = src.crs

# Create a point at the center of the raster
center_x = (bounds.left + bounds.right) / 2
center_y = (bounds.top + bounds.bottom) / 2
point = Point(center_x, center_y)

# Create GeoDataFrame
gdf = gpd.GeoDataFrame({'id': [1]}, geometry=[point], crs=crs)

print("Point within raster:", point)
print("CRS:", crs)


Point within raster: POINT (747960 -2804895)
CRS: EPSG:32621


## Extracting time series images by point geometry
We can also easily extract a time series of raster images. Extracted pixel values are provided in 'wide' format with appropriate labels, for instance the column 't2_blue' would be the blue band for the second time period

In [40]:
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point

# Path to your raster
l8_224078_20200518 = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"

# Create a valid point in the same CRS and within raster bounds
# We'll use a point near the center of the raster
point = Point(748000, -2800000)  # EPSG:32621
points_gdf = gpd.GeoDataFrame(
    {'id': [1], 'name': ['point1']}, 
    geometry=[point], 
    crs='EPSG:32621'
)

# Load raster twice to simulate time stacking
src1 = rxr.open_rasterio(l8_224078_20200518, chunks=True)
src2 = rxr.open_rasterio(l8_224078_20200518, chunks=True)

# Concatenate along a time dimension
src = xr.concat([src1, src2], dim='time')
src = src.assign_coords(time=['t1', 't2'])

# Clip using bounding box of the point
df = src.rio.clip_box(
    *points_gdf.total_bounds, 
    crs=points_gdf.crs,
    allow_one_dimensional_raster=True  # Optional but helps with point clip
)

print(df)


<xarray.DataArray (time: 2, band: 3, y: 1, x: 1)> Size: 12B
dask.array<getitem, shape=(2, 3, 1, 1), dtype=uint16, chunksize=(1, 1, 1, 1), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 24B 1 2 3
  * x            (x) float64 8B 7.48e+05
  * y            (y) float64 8B -2.8e+06
  * time         (time) <U2 16B 't1' 't2'
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0


## Extracting data by polygon geometry

To extract values within polygons, use the same `geowombat.extract` function.

In [41]:
import rioxarray as rxr
import geopandas as gpd
import rasterio
from shapely.geometry import Point
import pandas as pd

# Create sample points
points = [Point(748000, -2800000)]
l8_224078_20200518_points = gpd.GeoDataFrame(
    {'id': [1], 'name': ['point1']}, 
    geometry=points, 
    crs='EPSG:32633'
)

point_df = l8_224078_20200518_points.copy()
print(point_df.crs)
# Transform the CRS to WGS84 lat/lon
point_df = point_df.to_crs('epsg:4326')
print(point_df.crs)

# Extract values at point locations using rasterio sampling
results = []
with rasterio.open(l8_224078_20200518) as src:
    # Transform points back to raster CRS for sampling
    points_in_raster_crs = point_df.to_crs(src.crs)
    
    for idx, point in points_in_raster_crs.iterrows():
        geom = point.geometry
        if geom.geom_type == 'Point':
            row, col = src.index(geom.x, geom.y)
            if 0 <= row < src.height and 0 <= col < src.width:
                values = src.read()[:, row, col]
                result = {f'band_{i+1}': val for i, val in enumerate(values)}
                result.update({col: point[col] for col in point_df.columns if col != 'geometry'})
                results.append(result)

df = pd.DataFrame(results)
print(df)

EPSG:32633
epsg:4326
Empty DataFrame
Columns: []
Index: []


### Calculate mean pixel value by polygon
It is simple then to calculate the mean value of pixels within each polygon by using the polygon `id` column and pandas groupby function. You can easily calculate other statistics like min, max, median etc.

In [53]:
from shapely.geometry import box
import rioxarray as rxr
import geopandas as gpd
import numpy as np

# Load raster
raster_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"
src = rxr.open_rasterio(raster_path, chunks=True)
xmin, ymin, xmax, ymax = src.rio.bounds()
raster_crs = src.rio.crs

# Calculate valid inner box (using 20% reduction from edges)
width = xmax - xmin
height = ymax - ymin
buffer_x = width * 0.20  # Reduce 20% from left/right
buffer_y = height * 0.20  # Reduce 20% from top/bottom

# Create valid inner polygon
inner_box = box(
    xmin + buffer_x,
    ymin + buffer_y,
    xmax - buffer_x,
    ymax - buffer_y
)
polygons = gpd.GeoDataFrame(
    {'id': [1]}, 
    geometry=[inner_box], 
    crs=raster_crs
)

# Assign band names
band_names = [f'band_{i+1}' for i in range(len(src.band))]
src = src.assign_coords(band=band_names)

# Clip and compute mean (more efficient method)
clipped = src.rio.clip(polygons.geometry, polygons.crs)

# Compute mean directly without converting to DataFrame
band_means = clipped.mean(dim=['x', 'y'])

# Convert to DataFrame with proper naming
df_mean = band_means.to_dataframe(name='mean_value').reset_index()
df_mean = df_mean[['band', 'mean_value']]

print(df_mean)

     band   mean_value
0  band_1  7752.016690
1  band_2  7242.926096
2  band_3  6631.088370
